In [1]:
!pip install flask-ngrok

In [2]:
import os
import zipfile

In [3]:
!git clone https://github.com/sulc/tfrecord-viewer.git

Cloning into 'tfrecord-viewer'...
remote: Enumerating objects: 21, done.
remote: Total 21 (delta 0), reused 0 (delta 0), pack-reused 21
Unpacking objects: 100% (21/21), done.


In [4]:
rootPath = '/content/drive/My Drive/Machine Learning/Stolen vehicle detection'
dataPath = rootPath + '/data'

localPath = '/content'
localDataPath = localPath + '/data'

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

os.chdir(localPath)
# Show current directory
!pwd

Mounted at /content/drive
/content


In [6]:
# Run it only when the dataset is not yet extracted 
zipRef = zipfile.ZipFile(dataPath + "/datasetRecords.zip", 'r')
zipRef.extractall(localDataPath)
zipRef.close()
print('Dataset extraction successful')

Dataset extraction successful


In [7]:
train_record_fname = '/content/data/datasetRecords/trainDataset.tfrecord-?????-of-00012'
validation_record_fname = '/content/data/datasetRecords/validationDataset.tfrecord-?????-of-00001'
test_record_fname = '/content/data/datasetRecords/testDataset.tfrecord-?????-of-00001'
label_map_pbtxt_fname = '/content/data/datasetRecords/classes.pbtxt'

##Write file

In [17]:
%%writefile {'/content/tfrecord-viewer/tfviewer.py'}

#!/usr/bin/env python3
import sys
import io
import argparse

import tensorflow as tf
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, send_file

from overlays import overlay_factory

app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run

parser = argparse.ArgumentParser(description='TF Record viewer.')
parser.add_argument('tfrecords', type=str, nargs='+',
                    help='path to TF record(s) to view')

parser.add_argument('--image-key', type=str, default="image/encoded",
                    help='Key to the encoded image.')

parser.add_argument('--filename-key', type=str, default="image/filename",
                    help='Key to the unique ID of each record.')

parser.add_argument('--max-images', type=int, default=500,
                    help='Max. number of images to load.')

parser.add_argument('--host', type=str, default="0.0.0.0",
                    help='host/IP to start the Flask server.')

parser.add_argument('--port', type=int, default=5000,
                    help='Port to start the Flask server.')

parser.add_argument("-v", "--verbose", help="increase output verbosity",
                    action="store_true")

parser.add_argument('--overlay', type=str, default="detection",
                    help='Overlay to display. (detection/classification/none)')


#######################################
# Object detection specific arguments #
parser.add_argument('--bbox-name-key', type=str, default="image/object/class/text",
                    help='Key to the bbox label.')

parser.add_argument('--bbox-xmin-key', type=str, default="image/object/bbox/xmin")
parser.add_argument('--bbox-xmax-key', type=str, default="image/object/bbox/xmax")
parser.add_argument('--bbox-ymin-key', type=str, default="image/object/bbox/ymin")
parser.add_argument('--bbox-ymax-key', type=str, default="image/object/bbox/ymax")

parser.add_argument('--coordinates-in-pixels', action="store_true",
                    help='Set if bounding box coordinates are saved in pixels, not in %% of image width/height.')

parser.add_argument('--labels-to-highlight', type=str, default="vehicle",
                    help='Labels for which bounding boxes should be highlighted (orange instead of green).')


###########################################
# Image classification specific arguments #
parser.add_argument('--class-label-key', type=str, default="image/class/text",
                    help='Key to the image class label.')


args = parser.parse_args()


# Variables to be loaded with preload_images()
images = []
filenames = []
captions = []
bboxes = []


def preload_images(max_images):
  """ 
  Load images to be displayed in the browser gallery.

  Args:
    max_images (int): Maximum number of images to load.
  Returns:
    count (int): Number of images loaded.
  """
  count = 0
  overlay = overlay_factory.get_overlay(args.overlay, args)

  for tfrecord_path in args.tfrecords:
    print("Filename: ", tfrecord_path)
    for i, record in enumerate(tf.compat.v1.io.tf_record_iterator(tfrecord_path)):
      if args.verbose: print("######################### Record", i, "#########################")
      example = tf.train.Example()
      example.ParseFromString(record)
      feat = example.features.feature

      if len(images) < max_images:
        filename = feat[args.filename_key].bytes_list.value[0].decode("utf-8")
        img =  feat[args.image_key].bytes_list.value[0]
        
        img_with_overlay = overlay.apply_overlay(img, feat)

        filenames.append(filename)
        images.append(img_with_overlay)
        captions.append( tfrecord_path + ":" + filename )
      else:
        return count
      count += 1
  return count



@app.route('/')
def frontpage():
  html = ""
  for i,filename in enumerate(filenames):
    html += '<img data-u="image" src="image/%s" data-caption="%s" />\n' % (i, captions[i])
  return render_template('gallery.html', header=args.tfrecords, images=html)

@app.route('/image/<key>')
def get_image(key):
  """Get image by key (index) from images preloaded when starting the viewer by preload_images().
  """
  key=int(key)
  img = images[key]
  img_buffer = io.BytesIO(img)
  return send_file(img_buffer,
                   attachment_filename=str(key)+'.jpeg',
                   mimetype='image/jpg')

@app.after_request
def add_header(r):
  """
  Add headers to disable Caching,
  (So that images with the same index in different TFRecords are displayed correctly.)
  """
  r.headers["Cache-Control"] = "no-cache, no-store, must-revalidate"
  r.headers["Pragma"] = "no-cache"
  r.headers["Expires"] = "0"
  r.headers['Cache-Control'] = 'public, max-age=0'
  return r


if __name__ == "__main__":
  print("Pre-loading up to %d examples.." % args.max_images)
  count = preload_images(args.max_images)
  print("Loaded %d examples" % count)
  app.run()

Overwriting /content/tfrecord-viewer/tfviewer.py


In [21]:
%%writefile {'/content/tfrecord-viewer/overlays/detection_overlay.py'}

import io
from PIL import Image, ImageDraw, ImageFont


default_color = '#FF4500'
highlight_color = '#39FF14'
bbox_width = 4


class DetectionOverlay:
  
  def __init__(self, args):
    self.args = args
    self.labels_to_highlight = args.labels_to_highlight.split(";")
    self.font = ImageFont.truetype("./fonts/LiberationMono-Regular.ttf", 12)

  def apply_overlay(self, image_bytes, feature):
    """Apply annotation overlay over input image.
    
    Args:
      image_bytes: JPEG image.
      feature: TF Record Feature

    Returns:
      image_bytes_with_overlay: JPEG image with annotation overlay.
    """

    bboxes = self.get_bbox_tuples(feature)
    image_bytes_with_overlay = self.draw_bboxes(image_bytes, bboxes)
    return image_bytes_with_overlay


  def get_bbox_tuples(self, feature):
    """ From a TF Record Feature, get a list of tuples representing bounding boxes
    
    Args:
      feature: TF Record Feature
    Returns:
      bboxes (list of tuples): [ (label, xmin, xmax, ymin, ymax), (label, xmin, xmax, ymin, ymax) , .. ]
    """
    bboxes = []
    if self.args.bbox_name_key in feature:
      for ibbox, label in enumerate (feature[self.args.bbox_name_key].bytes_list.value):
        bboxes.append( (label.decode("utf-8"),
                        feature[self.args.bbox_xmin_key].float_list.value[ibbox],
                        feature[self.args.bbox_xmax_key].float_list.value[ibbox],
                        feature[self.args.bbox_ymin_key].float_list.value[ibbox],
                        feature[self.args.bbox_ymax_key].float_list.value[ibbox]
      ) )
    else:
      print("Bounding box key '%s' not present." % (self.args.bbox_name_key))
    return bboxes

  def bbox_color(self, label):
    if label in self.labels_to_highlight:
      return highlight_color
    else:
      return default_color

  def bboxes_to_pixels(self, bbox, im_width, im_height):
    """
    Convert bounding box coordinates to pixels.
    (It is common that bboxes are parametrized as percentage of image size
    instead of pixels.)

    Args:
      bboxes (tuple): (label, xmin, xmax, ymin, ymax)
      im_width (int): image width in pixels
      im_height (int): image height in pixels
    
    Returns:
      bboxes (tuple): (label, xmin, xmax, ymin, ymax)
    """
    if self.args.coordinates_in_pixels:
      return bbox
    else:
      label, xmin, xmax, ymin, ymax = bbox
      return [label, xmin * im_width, xmax * im_width, ymin * im_height, ymax * im_height]

  def draw_bboxes(self, image_bytes, bboxes):
    """Draw bounding boxes onto image.
    
    Args:
      image_bytes: JPEG image.
      bboxes (list of tuples): [ (label, xmin, xmax, ymin, ymax), (label, xmin, xmax, ymin, ymax) , .. ]
    
    Returns:
      image_bytes: JPEG image including bounding boxes.
    """
    img = Image.open(io.BytesIO(image_bytes))

    draw = ImageDraw.Draw(img)

    width, height = img.size

    for bbox in bboxes:
      label, xmin, xmax, ymin, ymax = self.bboxes_to_pixels(bbox, width, height)
      draw.rectangle([xmin, ymin, xmax, ymax], outline=self.bbox_color(label), width=bbox_width)

      w, h = self.font.getsize(label)
      draw.rectangle((xmin, ymin, xmin + w + 4, ymin + h), fill="white")

      draw.text((xmin+4, ymin), label, fill=self.bbox_color(label), font=self.font)

    with io.BytesIO() as output:
      img.save(output, format="JPEG")
      output_image = output.getvalue()
    return output_image

Overwriting /content/tfrecord-viewer/overlays/detection_overlay.py


##Run

In [25]:
!python3 /content/tfrecord-viewer/tfviewer.py '/content/data/datasetRecords/trainDataset.tfrecord-00002-of-00012' --labels-to-highlight='Vehicle registration plate'

2020-10-12 13:08:07.469519: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
Pre-loading up to 500 examples..
Filename:  /content/data/datasetRecords/trainDataset.tfrecord-00002-of-00012
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Loaded 500 examples
 * Serving Flask app "tfviewer" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Running on http://ebace1e0daaf.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
127.0.0.1 - - [12/Oct/2020 13:08:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2020 13:08:37] "GET /image/0 HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2020 13:08:37] "GET /image/1 HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2020 13:08:37] "GET /image/2 HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/2020 13:08:37] "GET /image/5 HTTP/1.1" 200 -
127.0.0.1 - - [12/Oct/